In [5]:
import sys
import time
import numpy as np
from copy import deepcopy

In [6]:
!ls

assignment2.pdf  __pycache__	       q2_test.predicted	  utils.py
data		 q1_classifier.py      q3_rnnlm.ipynb		  utils.pyc
data_utils	 q1_softmax.py	       q3_RNNLM.py		  weights
model.py	 q2_initialization.py  tensorflow_tutorial.ipynb
model.pyc	 q2_NER.py	       test.py


#### 导入本地函数

In [7]:
from utils import calculate_perplexity, get_ptb_dataset, Vocab
from utils import ptb_iterator, sample

In [8]:
import tensorflow as tf
from tensorflow.contrib.legacy_seq2seq.python.ops.seq2seq import sequence_loss
from model import LanguageModel

#### 参数配置
参数设置可以参考这篇文献　http://arxiv.org/pdf/1409.2329v4.pdf 

In [9]:
class Config(object):
    """
    Holds model hyperparams and data information.

    The config class is used to store various hyperparameters and dataset
    information parameters. Model objects are passed a Config() object at
    instantiation.
    """
    batch_size = 64
    embed_size = 50
    hidden_size = 100
    num_steps = 10
    max_epochs = 16
    early_stopping = 2 # ?? for what?
    dropout = 0.9
    lr = 0.001

In [10]:
from collections import defaultdict
word_freq = defaultdict(int)
print(word_freq)
word_freq['xie'] += 1
print(word_freq)

defaultdict(<class 'int'>, {})
defaultdict(<class 'int'>, {'xie': 1})


In [11]:
class RNNLM_Model(LanguageModel):
    def __init__(self, config):
        self.config = config
#         self.load_data(debug=False)
#         self.add_placeholders()
#         self.inputs = self.add_embedding()
#         self.rnn_outputs = self.add_model(self.inputs)
#         self.outputs = self.add_projection(self.rnn_outputs)

#         # We want to check how well we correctly predict the next word
#         # We cast o to float64 as there are numerical issues at hand
#         # (i.e. sum(output of softmax) = 1.00000298179 and not 1)
#         self.predictions = [tf.nn.softmax(tf.cast(o, 'float64')) for o in self.outputs]
#         # Reshape the output into len(vocab) sized chunks - the -1 says as many as
#         # needed to evenly divide
#         output = tf.reshape(tf.concat(1, self.outputs), [-1, len(self.vocab)])
#         self.calculate_loss = self.add_loss_op(output)
#         self.train_step = self.add_training_op(self.calculate_loss)

In [12]:
config = Config()
model = RNNLM_Model(config)

### 获取数据

In [13]:
def load_data(self, debug=False):
    """
    Loads starter word-vectors and train/dev/test data.
    """
    self.vocab = Vocab() # 新建一个实例类
    self.vocab.construct(get_ptb_dataset('train'))
    self.encoded_train = np.array(
        [self.vocab.encode(word) for word in get_ptb_dataset('train')],
        dtype=np.int32)
    self.encoded_valid = np.array(
        [self.vocab.encode(word) for word in get_ptb_dataset('valid')],
        dtype=np.int32)
    self.encoded_test = np.array(
        [self.vocab.encode(word) for word in get_ptb_dataset('test')],
        dtype=np.int32)
    if debug:
        num_debug = 1024
        self.encoded_train = self.encoded_train[:num_debug]
        self.encoded_valid = self.encoded_valid[:num_debug]
        self.encoded_test = self.encoded_test[:num_debug]

#### 动态给类增加方法，以便于查看～

In [14]:
model.load_data = load_data(model, debug=False)
print(dir(model))
# alternate


929589.0 total words with 10000 uniques
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'add_embedding', 'add_loss_op', 'add_model', 'add_placeholders', 'config', 'create_feed_dict', 'encoded_test', 'encoded_train', 'encoded_valid', 'fit', 'load_data', 'predict', 'run_epoch', 'vocab']


In [41]:
print(len(model.vocab))

10000


In [16]:
print(model.encoded_train[30:40])
print(model.encoded_valid[:10])
print(model.encoded_test[:10])

[30 31 32 33 34 35 36 37 38 27]
[2211  396 1129   64  831  556  808   35 1382 2745]
[ 142   78   54  251 2360  405   25  315  706   32]


### 添加placeholder

In [17]:
def add_placeholders(self):
    self.input_placeholder = tf.placeholder(dtype=tf.int32, shape =[None, self.config.num_steps], name='input')
    self.labels_placeholder = tf.placeholder(dtype=tf.float32, shape=[None, self.config.num_steps], name='labels')
    self.dropout_placeholder = tf.placeholder(dtype=tf.float32,name='dropout')

#### 动态增加方法

In [18]:
model.add_placeholders = add_placeholders(model)

### 添加embedding层

In [19]:
def add_embedding(self):
    # The embedding lookup is currently only implemented for the CPU
    with tf.device('/cpu:0'):
        with tf.variable_scope('embedding_layer'):
            embedding = tf.get_variable("embedding",
                                        shape=[len(self.vocab), self.config.embed_size],
                                        initializer=tf.random_uniform_initializer(-1,1),
                                        trainable=True)
        # 按照input_placeholder中的index提取词向量
        inputs = tf.nn.embedding_lookup(params=embedding,
                                        ids = self.input_placeholder) 
        inputs = tf.split(value=inputs,
                          num_or_size_splits=self.config.num_steps,
                          axis=0)
        for i in range(len(inputs)):
            inputs[i] = tf.squeeze(inputs[i], [1])
    return inputs

In [40]:
model.add_embedding = add_embedding(model)

ValueError: Can not squeeze dim[1], expected a dimension of 1, got 10 for 'Squeeze' (op: 'Squeeze') with input shapes: [?,10,50].

#### tf.nn.embedding_lookup(params, ids)

In [39]:
embedding= tf.Variable(tf.random_uniform((100,5), -1, 1), name="embeddings")
# ids = self.input_placeholder
inputs = tf.nn.embedding_lookup(params=embedding, ids=[0,1]) # ids
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    print(sess.run(embedding[:5]))
    print(inputs)
    inputs = tf.split(inputs, num_or_size_splits=2,axis=0)
    print(inputs)

[[ 0.41319418  0.96770215 -0.28230405 -0.08141661  0.63735294]
 [-0.24879193  0.06770301  0.25892806 -0.7525084   0.21936822]
 [ 0.25886106 -0.73741126 -0.84952354 -0.54619741 -0.2374115 ]
 [-0.45024443  0.31670499  0.47285414  0.48613834 -0.056144  ]
 [ 0.83776236  0.04529333 -0.96767926  0.45418692  0.45970273]]
Tensor("embedding_lookup_6:0", shape=(2, 5), dtype=float32)
[<tf.Tensor 'split_1:0' shape=(1, 5) dtype=float32>, <tf.Tensor 'split_1:1' shape=(1, 5) dtype=float32>]


#### ValueError: Variable embedding already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

使用tf.get_variable 容易出现这样的错误。
要理解tf.Variable, tf.get_variable,以及变量管理name_scope, variable_scope的区别，可以查看这篇[blog](https://blog.csdn.net/Jerr__y/article/details/70809528)